# Epic 5:

Als een key user kan ik voor een campagne een lijst met contacten genereren volgens de waarschijnlijkheid om in te schrijven voor de campagne. Ik wil in de sortering ook ervoor zorgen dat contacten met weinig marketing pressure bevoordeeld worden ten opzichte van contacten met een hoge marketing pressure. 

In [2]:
# Importing the necessary packages
import numpy as np                                  # "Scientific computing"
import scipy.stats as stats                         # Statistical tests

import pandas as pd                                 # Data Frame
from pandas.api.types import CategoricalDtype

import matplotlib.pyplot as plt                     # Basic visualisation
from statsmodels.graphics.mosaicplot import mosaic  # Mosaic diagram
import seaborn as sns   

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics.pairwise import cosine_similarity

## Data Cleaning

In [42]:
campange_df = pd.read_csv("Data/Campagne.csv")
campanges_df = pd.read_csv('Data/Campagnes.csv')

campange_df.set_index("crm_Campagne_Campagne")
campanges_df.set_index("crm_Campagne_Campagne")


df = pd.merge(campange_df, campanges_df, how="outer")
df.set_index("crm_Campagne_Campagne")



,crm_Campagne_Campagne_Nr,crm_Campagne_Einddatum,crm_Campagne_Naam,crm_Campagne_Naam_in_email,crm_Campagne_Reden_van_status,crm_Campagne_Startdatum,crm_Campagne_Status,crm_Campagne_Type_campagne,crm_Campagne_URL_voka_be,crm_Campagne_Soort_Campagne
crm_Campagne_Campagne,,,,,,,,,,
002B50B9-36D9-E911-8105-001DD8B72B61,19-OV-01-0197,16-9-2019 15:00:00,OV-NW-Voka Update-AIO-Sep2019-Een nieuw vennoo...,OV-NW-Voka Update-AIO-Sep2019-Een nieuw vennoo...,Voltooid,16-9-2019 14:00:00,Actief,Netwerkevenement,NaN,Offline
03418937-184E-ED11-BBA2-6045BD8952CE,22-OV-01-0172,10-2-2023 14:30:00,OV-MA/NW-Regiolunch-Zaken die smaken-Meetjesland,Regiolunch-Zaken die smaken Meetjesland,Voltooid,10-2-2023 12:00:00,Actief,Netwerkevenement,https://www.voka.be/activiteiten/regiolunch-za...,Offline
037E70BA-6D2F-E911-80FB-001DD8B72B62,19-OV-01-0047,27-3-2019 15:00:00,OV-Match-Lunchmeeting Sofie Bracke - Maart 2019,Lunchmeeting scale-ups vs Sofie Bracke,Voltooid,27-3-2019 12:00:00,Actief,Netwerkevenement,NaN,Offline
042E13FF-D097-ED11-AAD1-6045BD895B5A,23-OV-01-0043,6-2-2023 10:00:00,OV-NW-AIO-EDIH-DIGITALIS-Voka Update-Big Refre...,OV-NW-AIO-EDIH-DIGITALIS-Voka Update-Big Refre...,Voltooid,6-2-2023 08:30:00,Actief,Netwerkevenement,NaN,Offline
05FB51E2-7564-EB11-811B-001DD8B72B62,21-OV-01-0020,11-2-2021 12:30:00,OV-NW-AIO-Voka Update-Big Refresh-Voorjaar 202...,OV-NW-AIO-Voka Update-Big Refresh-Voorjaar 202...,Voltooid,11-2-2021 11:30:00,Actief,Netwerkevenement,NaN,Online
...,...,...,...,...,...,...,...,...,...,...
FFD1A747-FE02-EB11-8114-001DD8B72B61,20-OV-06-0048,20-11-2020 11:00:00,OV-Webinar Brexit,OV-Webinar Brexit,Nieuw,16-10-2020 09:00:00,Actief,Infosessie,NaN,Online
FFDED034-7A73-E711-80EB-001DD8B72B61,17-OV-02-0119,6-3-2018 20:00:00,OV-Expert Class - IT managers,Expert Class - IT managers,Nieuw,10-10-2017 06:00:00,Actief,Opleiding,NaN,Offline
FFE22A80-3BF3-E111-8D2C-984BE17C47FB,12-OV-01-0074,NaN,OV-NW-Bezoek Track Jong Voka,OV-NW-Bezoek Track Jong Voka,Nieuw,13-9-2012 00:00:00,Actief,Netwerkevenement,NaN,Offline


In [ ]:
from sklearn.model_selection import train_test_split

# train_set, test_set = train_test_split(df, test_size=0.2, random_state=42)

In [23]:
# campange_df.info() # 168 entries before cleaning
# campanges_df.info() # not every column even amount of entries

/tmp/ipykernel_10306/1775109848.py:4: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  campanges_df.fillna(method="ffill", inplace=True)


In [13]:
# campange_df.dropna(inplace=True)
# campanges_df.dropna(inplace=True)

In [24]:
# campange_df.info() # 53 after cleaning
# campanges_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4092 entries, 0 to 4091
Data columns (total 11 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   crm_Campagne_Campagne          4092 non-null   object
 1   crm_Campagne_Campagne_Nr       4092 non-null   object
 2   crm_Campagne_Einddatum         4092 non-null   object
 3   crm_Campagne_Naam              4092 non-null   object
 4   crm_Campagne_Naam_in_email     4092 non-null   object
 5   crm_Campagne_Reden_van_status  4092 non-null   object
 6   crm_Campagne_Startdatum        4092 non-null   object
 7   crm_Campagne_Status            4092 non-null   object
 8   crm_Campagne_Type_campagne     4092 non-null   object
 9   crm_Campagne_URL_voka_be       4087 non-null   object
 10  crm_Campagne_Soort_Campagne    4092 non-null   object
dtypes: object(11)
memory usage: 351.8+ KB


In [35]:
campanges_df.value_counts()

crm_Campagne_Campagne                 crm_Campagne_Campagne_Nr  crm_Campagne_Einddatum  crm_Campagne_Naam                                                       crm_Campagne_Naam_in_email                                               crm_Campagne_Reden_van_status  crm_Campagne_Startdatum  crm_Campagne_Status  crm_Campagne_Type_campagne  crm_Campagne_URL_voka_be                                                                            crm_Campagne_Soort_Campagne
003E6D68-2D71-EB11-8119-001DD8B72B61  21-OV-02-0090             26-4-2022 19:00:00      OV-IN-Digitaal transformeren voor KMO's - AIO Langlopend traject        Digitaal transformeren voor KMO's                                        Nieuw                          17-6-2021 15:00:00       Actief               Opleiding                   https://www.voka.be/activiteiten/oost-vlaanderen-hoe-cyberveilig-digitaal-transformeren-met-uw-kmo  Offline                        1
AD0DBA5F-3ED0-E711-80EC-001DD8B72B62  18-OV-02-0014            

In [45]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4092 entries, 0 to 4091
Data columns (total 11 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   crm_Campagne_Campagne          4092 non-null   object
 1   crm_Campagne_Campagne_Nr       4067 non-null   object
 2   crm_Campagne_Einddatum         3782 non-null   object
 3   crm_Campagne_Naam              4092 non-null   object
 4   crm_Campagne_Naam_in_email     4092 non-null   object
 5   crm_Campagne_Reden_van_status  4092 non-null   object
 6   crm_Campagne_Startdatum        4055 non-null   object
 7   crm_Campagne_Status            4092 non-null   object
 8   crm_Campagne_Type_campagne     4092 non-null   object
 9   crm_Campagne_URL_voka_be       852 non-null    object
 10  crm_Campagne_Soort_Campagne    4092 non-null   object
dtypes: object(11)
memory usage: 351.8+ KB


In [60]:
df["crm_Campagne_Einddatum"] = pd.to_datetime(df["crm_Campagne_Einddatum"])
df["crm_Campagne_Startdatum"] = pd.to_datetime(df["crm_Campagne_Startdatum"])

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4092 entries, 0 to 4091
Data columns (total 11 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   crm_Campagne_Campagne          4092 non-null   object        
 1   crm_Campagne_Campagne_Nr       4067 non-null   object        
 2   crm_Campagne_Einddatum         3782 non-null   datetime64[ns]
 3   crm_Campagne_Naam              4092 non-null   object        
 4   crm_Campagne_Naam_in_email     4092 non-null   object        
 5   crm_Campagne_Reden_van_status  4092 non-null   object        
 6   crm_Campagne_Startdatum        4055 non-null   datetime64[ns]
 7   crm_Campagne_Status            4092 non-null   object        
 8   crm_Campagne_Type_campagne     4092 non-null   object        
 9   crm_Campagne_URL_voka_be       852 non-null    object        
 10  crm_Campagne_Soort_Campagne    4092 non-null   object        
dtypes: datetime64[ns]

/tmp/ipykernel_10306/2405451660.py:2: UserWarning: Parsing dates in %d-%m-%Y %H:%M:%S format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df["crm_Campagne_Startdatum"] = pd.to_datetime(df["crm_Campagne_Startdatum"])


In [61]:
df.head()

,crm_Campagne_Campagne,crm_Campagne_Campagne_Nr,crm_Campagne_Einddatum,crm_Campagne_Naam,crm_Campagne_Naam_in_email,crm_Campagne_Reden_van_status,crm_Campagne_Startdatum,crm_Campagne_Status,crm_Campagne_Type_campagne,crm_Campagne_URL_voka_be,crm_Campagne_Soort_Campagne
0,002B50B9-36D9-E911-8105-001DD8B72B61,19-OV-01-0197,2019-09-16 15:00:00,OV-NW-Voka Update-AIO-Sep2019-Een nieuw vennoo...,OV-NW-Voka Update-AIO-Sep2019-Een nieuw vennoo...,Voltooid,2019-09-16 14:00:00,Actief,Netwerkevenement,NaN,Offline
1,03418937-184E-ED11-BBA2-6045BD8952CE,22-OV-01-0172,2023-02-10 14:30:00,OV-MA/NW-Regiolunch-Zaken die smaken-Meetjesland,Regiolunch-Zaken die smaken Meetjesland,Voltooid,2023-02-10 12:00:00,Actief,Netwerkevenement,https://www.voka.be/activiteiten/regiolunch-za...,Offline
2,037E70BA-6D2F-E911-80FB-001DD8B72B62,19-OV-01-0047,2019-03-27 15:00:00,OV-Match-Lunchmeeting Sofie Bracke - Maart 2019,Lunchmeeting scale-ups vs Sofie Bracke,Voltooid,2019-03-27 12:00:00,Actief,Netwerkevenement,NaN,Offline
3,042E13FF-D097-ED11-AAD1-6045BD895B5A,23-OV-01-0043,2023-02-06 10:00:00,OV-NW-AIO-EDIH-DIGITALIS-Voka Update-Big Refre...,OV-NW-AIO-EDIH-DIGITALIS-Voka Update-Big Refre...,Voltooid,2023-02-06 08:30:00,Actief,Netwerkevenement,NaN,Offline
4,05FB51E2-7564-EB11-811B-001DD8B72B62,21-OV-01-0020,2021-02-11 12:30:00,OV-NW-AIO-Voka Update-Big Refresh-Voorjaar 202...,OV-NW-AIO-Voka Update-Big Refresh-Voorjaar 202...,Voltooid,2021-02-11 11:30:00,Actief,Netwerkevenement,NaN,Online


## Data Preprocessing


### Handling Text and Categorical Attributes

In [47]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

cat_attribs = ["crm_Campagne_Reden_van_status","crm_Campagne_Status",  "crm_Campagne_Type_campagne", "crm_Campagne_Soort_Campagne"]


cat_pipeline = Pipeline([
    ("impute", SimpleImputer(strategy="most_frequent")),
    ("encoder", OrdinalEncoder())
])

prep = ColumnTransformer([
    ('cat', cat_pipeline, cat_attribs)
])

df_prep = prep.fit_transform(df)


In [64]:
df_prep
df_prep.shape

(4092, 4)

## Select and Train a Model

#### Soort Model:

- Recommender system 
- Model marketing pressure bepalen:
  - UNKNOWN